# Практика 8: Методы защиты от атак на модели ИИ
## Выполнил студент Благоразумов Александр Сергеевич группы ББМО-01-23

Шаг 1: Защита с помощью Adversarial Training

Adversarial Training — это метод защиты, при котором модель обучается на специально созданных противоречивых примерах. Такой подход повышает устойчивость модели к атакам, поскольку она изначально адаптируется к работе с подобными примерами в процессе обучения.

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Уменьшаем количество обучающих изображений до 1000
train_images = train_images[:1000]
train_labels = train_labels[:1000]

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * np.sign(gradient)
    adversarial_image = image + perturbation
    adversarial_image = np.clip(adversarial_image, 0, 1)  # Ограничение значений пикселей
    return adversarial_image

# Функция для генерации противоречивых примеров
def generate_adversarial_examples(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)), dtype=tf.float32)  # Преобразование в tf.Tensor
        label = tf.convert_to_tensor(labels[i].reshape((1, 10)), dtype=tf.float32)  # Преобразование метки в Tensor

        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)
        gradient = tape.gradient(loss, image)
        adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())  # Конвертируем обратно в numpy
        adversarial_images.append(adversarial_image.reshape(28, 28))  # Преобразуем в форму (28, 28)

    return np.array(adversarial_images)

# Создание модели
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с противоречивыми примерами
def adversarial_training(model, train_images, train_labels, epsilon):
    for epoch in range(5):  # Короткое обучение для демонстрации
        for i in range(0, len(train_images), 32):
            batch_images = train_images[i:i+32]
            batch_labels = train_labels[i:i+32]
            # Генерация противоречивых примеров для текущей партии данных
            adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
            # Объединение оригинальных и противоречивых примеров
            combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
            combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
            # Обучение на комбинированных данных
            model.train_on_batch(combined_images, combined_labels)

# Инициализация модели
model = create_model()

# Тренировка модели с защитой на противоречивых примерах
adversarial_training(model, train_images, train_labels, epsilon=0.1)

# Сохранение защищенной модели
model.save('trained_model_adversarial.h5')

Шаг 2: Градиентная маскировка (Gradient Masking)

Gradient Masking — это метод защиты, который ограничивает доступ к градиентам модели, затрудняя проведение атак. Его цель — уменьшить объем информации, доступной для злоумышленников, и усложнить процесс нахождения направленных изменений.

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Загрузка и подготовка данных
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Обновление модели для градиентной маскировки
def create_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),
        Activation('softplus')  # Используем softplus вместо softmax для градиентной маскировки
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с градиентной маскировкой
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с градиентной маскировкой
masked_model.save('masked_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8772 - loss: 0.4627
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9630 - loss: 0.1308
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9738 - loss: 0.0872
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9808 - loss: 0.0624
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9847 - loss: 0.0502


Шаг 3: Регуляризация и нормализация для повышения устойчивости

In [4]:
# Модель с регуляризацией и нормализацией
def create_regularized_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu',
kernel_regularizer=tf.keras.regularizers.l2(0.01)),
 tf.keras.layers.Dropout(0.5),tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(10, activation='softmax')
 ])
 model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])
 return model

# Обучение модели с регуляризацией и нормализацией
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с регуляризацией
regularized_model.save('regularized_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7960 - loss: 1.3319
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8692 - loss: 0.6301
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8667 - loss: 0.6194
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8642 - loss: 0.6194
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8618 - loss: 0.6146


Шаг 4: Оценка моделей на противоречивых примерах

In [6]:
# Загрузка атакованной модели
protected_model = tf.keras.models.load_model('trained_model_adversarial.h5')

# Генерация противоречивых примеров для тестовых данных
adversarial_test_images = generate_adversarial_examples(protected_model,
test_images, test_labels, epsilon=0.1)

# Оценка защищенной модели
test_loss, test_acc = protected_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of protected model on adversarial examples: {test_acc}')

# Оценка модели с градиентной маскировкой
test_loss, test_acc = masked_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of masked model on adversarial examples: {test_acc}')

# Оценка модели с регуляризацией и нормализацией
test_loss, test_acc = regularized_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of regularized model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5010 - loss: 1.4502
Accuracy of protected model on adversarial examples: 0.5559999942779541
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7155 - loss: 0.9854
Accuracy of masked model on adversarial examples: 0.7455000281333923
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6352 - loss: 1.2214
Accuracy of regularized model on adversarial examples: 0.671500027179718


Вывод:


Предполагалось, что защитные методы, такие как Adversarial Training и Gradient Masking, смогут повысить точность моделей на противоречивых примерах. Регуляризация и нормализация также могут способствовать увеличению устойчивости модели к атакам.  

Модель с применением защиты продемонстрировала хороший результат, но есть потенциал для его улучшения. Модель с использованием градиентной маскировки показала более высокую устойчивость к атакам на противоречивых примерах по сравнению с обычной защищенной моделью.  

Наилучший результат продемонстрировала модель, использующая регуляризацию и нормализацию. При дальнейшем улучшении или сочетании защитных методов такая модель может значительно повысить свою устойчивость к атакам и улучшить ключевые метрики.